# Statistical analysis of hospital occupancy

In [18]:
# Import
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px


In [4]:
# Load data
hospitals = pd.read_csv('../data/processed/hospitals_collated_reduced.csv', encoding="utf-8")
hospitals.info()
hospitals.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                101 non-null    int64  
 1   KT                        101 non-null    object 
 2   Inst                      101 non-null    object 
 3   Adr                       100 non-null    object 
 4   Ort                       101 non-null    object 
 5   latitude                  91 non-null     float64
 6   longitude                 91 non-null     float64
 7   care_days                 101 non-null    float64
 8   max_capacity_in_bed_days  101 non-null    float64
 9   bed_occupancy             101 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 8.0+ KB


,Unnamed: 0,KT,Inst,Adr,Ort,latitude,longitude,care_days,max_capacity_in_bed_days,bed_occupancy
0,0,AG,Kantonsspital Aarau AG,Tellstrasse 25,5000 Aarau,47.388377,8.060297,140580.0,493.989041,77.967455
1,1,AG,Kantonsspital Baden AG,Im Ergel,5404 Baden,47.455266,8.282053,116437.0,364.000000,87.638868
2,2,AG,Hirslanden Klinik Aarau,Schänisweg 1,5000 Aarau,47.391175,8.041929,43434.0,155.000000,76.772426
3,3,AG,Gesundheitszentrum Fricktal,Riburgerstrasse 12,4310 Rheinfelden,47.556116,7.807022,37558.0,128.860274,79.852872
4,4,AG,Stiftung Spital Muri,Spitalstrasse 144,5630 Muri AG,47.277816,8.329914,32038.0,101.000000,86.906280


### Occupancy statistics over all hospitals
Use a violin plot to show the occupancy over all hospitals. Draw the recommended occupancy band in red.

In [56]:
import plotly.express as px

fig = px.box(hospitals, y='bed_occupancy', width=300, height=600,)

fig.add_hrect(y0=80, y1=92.5, fillcolor="lightcoral", opacity=0.6, layer="below", line_width=0,)

fig.show()

### Calculate statistics based on Canton
Calculate occupancy statistics for each canton and see whether they align with a utilization minimum of 88%.

In [17]:
cantonal_occ = hospitals.groupby(['KT'])['bed_occupancy'].agg(['mean', 'count', 'std']).reset_index()

cantonal_occ['tvalue'] = stats.t.ppf(0.975, cantonal_occ['count'] - 1)
cantonal_occ['me'] = cantonal_occ['tvalue'] * cantonal_occ['std'] / np.sqrt(cantonal_occ['count'])
cantonal_occ['ci_low'] = cantonal_occ['mean'] - cantonal_occ['me']
cantonal_occ['ci_high'] = cantonal_occ['mean'] + cantonal_occ['me']

cantonal_occ.head()

,KT,mean,count,std,tvalue,me,ci_low,ci_high
0,AG,84.746773,8,6.028227,2.364624,5.039724,79.707049,89.786497
1,AR,86.488169,1,NaN,NaN,NaN,NaN,NaN
2,BE,83.449548,12,10.473642,2.200985,6.654635,76.794913,90.104182
3,BL,84.910454,2,3.941649,12.706205,35.414311,49.496143,120.324766
4,BS,83.265282,3,6.879444,4.302653,17.089485,66.175796,100.354767


In [53]:
fig = px.bar(cantonal_occ, x='KT', y='mean', error_y='me', width=900, height=600,)

fig.update_yaxes(range=[50, 100])

fig.update_layout(
    uniformtext_minsize=16,
    uniformtext_mode='hide',
    title={
        'text': "Mean Bed Occupancy by Canton",
        'font': {'size': 24}
    },
    legend={
        'font': {'size': 16}
    },
    xaxis={'categoryorder': 'total descending'},
    xaxis_title="Canton",
    yaxis_title="Mean Bed Occupancy"
)

fig.add_hrect(y0=80, y1=92.5, fillcolor="lightcoral", opacity = 0.6, layer="below", line_width=0)

fig.add_annotation(x=0.5, y=91, text="Recommended occupancy", font_size=14,
                   showarrow=False, xshift=67)

fig.show()